In [2]:
import pandas as pd
import numpy as np
import time

# 读取女性评分
with open("Scores_for_Women.txt", "r") as f:
    w_to_m_data = f.readlines()[1:]

# 转化为矩阵 w_to_m_score_matrix[i][j] 为 女性i对男性j的评分
w_to_m_Scores = []    
for i in range(len(w_to_m_data)):
    w_to_m_Scores.append([float(ii) for ii in w_to_m_data[i].strip('\n').split(' ')[:-1]])
    
w_to_m_score_matrix = np.array(w_to_m_Scores)

# 读取女性评分
with open("Scores_for_Men.txt", "r") as f:
    m_to_w_data = f.readlines()[1:]

# 转化为矩阵 m_to_w_score_matrix[i][j] 为 男性i对女性j的评分    
m_to_w_Scores = []    
for i in range(len(m_to_w_data)):
    m_to_w_Scores.append([float(ii) for ii in m_to_w_data[i].strip('\n').split(' ')[:-1]])
    
m_to_w_score_matrix = np.array(m_to_w_Scores)


# 读取拒绝列表
with open("Reject_List.txt", "r") as f:
    reject_data = f.readlines()[1:]

# 转化为列表 
reject_list = []    
for i in range(len(reject_data)):
    reject_list.append([int(ii) for ii in reject_data[i].strip('\n').split(' ')])
    
# print('reject_list', reject_list)

W_dict = {} # 女性可接受词典
M_dict = {} # 男性可接受词典

for i in range(68):
    W_dict[i] = [i for i in range(68)]
    M_dict[i] = [i for i in range(68)]

for j in reject_list: # 按拒绝列表移除不接受的用户
    W_dict[j[0]].remove(j[1])
    M_dict[j[1]].remove(j[0])


In [3]:
# 使用调和平均数计算 双向评分
weight_matrix = 2/(1/(m_to_w_score_matrix.T) + 1/w_to_m_score_matrix) # weight_matrix[0][1] score between women 1 and men 2

# 计算 模型需要使用的参数

In [4]:
# 根据单向分数矩阵  计算 女性用户 i 在 男性用户 j 中的排名 和 男性用户 j 在 女性用户 i 中的排名

def find_max_position(input_array):
    # 查找数组中最大的数的坐标（可能是多个）
    # 如最大为 -1 则输出 [-1]
    max_number = max(input_array)
    if int(max_number) == -1:
        return [-1]
    index = np.where(max_number == input_array)[0]
    return index

w_m_score = w_to_m_score_matrix.copy()
m_w_score = m_to_w_score_matrix.copy()

w_to_m_rank = np.zeros((68,68))
m_to_w_rank = np.zeros((68,68))



for i in range(68):
    rank = 1
    while rank < 68:

        index_save = find_max_position(w_m_score[i])
#         print(index_save)
        if -1 in index_save:
            break

        for ii in index_save:
            w_m_score[i][ii] = -1
            w_to_m_rank[i][ii] = rank

        rank += 1

        
for i in range(68):
    rank = 1
    while rank < 68:

        index_save = find_max_position(m_w_score[i])
#         print(index_save)
        if -1 in index_save:
            break

        for ii in index_save:
            m_w_score[i][ii] = -1
            m_to_w_rank[i][ii] = rank

        rank += 1

In [5]:
m_to_w_rank[0] # 男性 0 的女性用户排名

array([12.,  9., 12.,  9.,  3.,  7., 21.,  2.,  9., 15.,  9.,  7., 11.,
       12.,  4.,  7.,  6.,  9., 16., 26.,  9.,  9., 10., 20., 25.,  9.,
        2.,  5.,  9.,  9.,  9.,  7.,  7., 12.,  7., 12., 14.,  7., 14.,
        7.,  2.,  7., 19., 16., 23., 26.,  8., 18.,  9., 12., 23.,  2.,
       12.,  9., 12., 14.,  2., 16., 22., 17.,  1.,  9.,  9., 26.,  9.,
       15., 24., 13.])

In [6]:
w_to_m_rank[0]

array([29., 25., 35.,  2., 22.,  4.,  4., 18., 34., 26., 13., 11.,  1.,
       22., 41., 20., 28., 24., 15.,  1., 31.,  4., 10., 15., 20.,  8.,
       15., 32., 34.,  2., 33., 17., 19., 20.,  6.,  4., 26., 12., 27.,
       24., 14., 21., 22.,  7., 16., 28., 36., 37., 25., 28., 20.,  5.,
       15., 40.,  3., 15., 18., 39., 20.,  9., 15., 30., 25.,  4., 39.,
       23., 26., 38.])

# 计算 “更喜欢或漠然集合”

In [7]:
w_to_m_better_or_indiff_than = [[[]for i in range(68)] for j in range(68)] # empty

# women to men
for i in range(68): # women i
    for j in range(68): # men j
        rank = w_to_m_rank[i][j]
        set_of_men = list(np.where(w_to_m_rank[i] <= rank)[0])
        w_to_m_better_or_indiff_than[i][j] = set_of_men
        
for r in reject_list:
    for i in range(68):
        if r[0] == i:
            for j in range(68):
                if r[1] in w_to_m_better_or_indiff_than[i][j]:
                    w_to_m_better_or_indiff_than[i][j].remove(r[1])

In [8]:
w_to_m_better_or_indiff_than[0][3] # the set of men that woman 0 ranks at the same level or better than man 3

[3, 12, 19, 29]

In [9]:
m_to_w_better_or_indiff_than = [[[]for i in range(68)] for j in range(68)] # empty
# men to women
for i in range(68): # men i
    for j in range(68): # women j
        rank = m_to_w_rank[i][j]
        set_of_women = list(np.where(m_to_w_rank[i] <= rank)[0])
        m_to_w_better_or_indiff_than[i][j] = set_of_women
        
for r in reject_list:
    for i in range(68):
        if r[1] == i:
            for j in range(68):
                if r[0] in m_to_w_better_or_indiff_than[i][j]:
                    m_to_w_better_or_indiff_than[i][j].remove(r[0])

In [10]:
m_to_w_better_or_indiff_than[0][5] # the set of women that man 0 ranks at the same level or better than woman 5

[4, 7, 11, 14, 15, 16, 26, 27, 31, 32, 34, 37, 39, 40, 41, 51, 56, 60]

# MIP SOLVER

# Assignment Problem, MAX-WT-SMTI-GRP

In [11]:
from mip import *



def MIP_Solver(Optimize_Direction = 'MAXIMIZE', Additional_Const = 'stable', Show_infor = False, OF = 'Maximize Weight'):
    # 最大化权重模型（无稳定性）：Optimize_Direction = 'MAXIMIZE', Additional_Const = 'unstable', OF = 'Maximize Weight' 
    # 稳定性模型：
    # 1.最大化权重：Optimize_Direction = 'MAXIMIZE', Additional_Const = 'stable', OF = 'Maximize Weight' 
    # 2.最小化总rank：Optimize_Direction = 'MINIMIZE', Additional_Const = 'stable', OF = 'Egalitarian'  
    
    
    if Optimize_Direction == 'MAXIMIZE':
        
        SMTI = Model(sense=MAXIMIZE, solver_name=GRB)
        
    elif Optimize_Direction == 'MINIMIZE':
        
        SMTI = Model(sense=MINIMIZE, solver_name=GRB)
        
    num_female = 68
    num_male = 68

    I = set(range(num_female))
    J = set(range(num_male))

    x = [[SMTI.add_var(var_type=BINARY) for j in J] for i in I] # Decision Variable

    if OF == 'Maximize Weight':
        SMTI.objective = xsum(weight_matrix[i][j]*x[i][j] for i in I for j in W_dict[i])
    elif OF == 'Egalitarian':
        SMTI.objective = xsum((m_to_w_rank[j][i] + w_to_m_rank[i][j])*x[i][j] for i in I for j in W_dict[i])

    for i in I:
        SMTI += xsum(x[i][j] for j in W_dict[i]) <= 1 

    for j in J:
        SMTI += xsum(x[i][j] for i in M_dict[j]) <= 1    

    if Additional_Const == 'stable':
        for i in I:
            for j in W_dict[i]:
                SMTI += 1 - xsum(x[i][q] for q in w_to_m_better_or_indiff_than[i][j]) <= xsum(x[p][j] for p in m_to_w_better_or_indiff_than[j][i])

    start = time.perf_counter()
    status = SMTI.optimize(max_seconds=600)
    end = time.perf_counter()
    runningtime = end-start
    solution = []
    
    if Show_infor == True:
        print('optimal solution score {} found'.format(SMTI.objective_value))
        print('Best Possible:', SMTI.objective_bound)
        print('model has {} vars, {} constraints and {} nzs'.format(SMTI.num_cols, SMTI.num_rows, SMTI.num_nz))
    
    for i in I:
        for j in J:
            if x[i][j].x > 0.99:
                solution.append([i,j])
    
    print('runningtime',runningtime)
    
    return solution, SMTI.objective_value # 匹配结果，模型最优值

In [12]:
MIP_Solver(Optimize_Direction = 'MAXIMIZE', Additional_Const = 'unstable',Show_infor=True, OF = 'Maximize Weight')[0] # 最大化权重

optimal solution score 3295.035193470071 found
Best Possible: 3295.0351934700716
model has 4624 vars, 136 constraints and 8844 nzs
runningtime 0.04110990000026504


[[0, 12],
 [1, 31],
 [2, 52],
 [3, 65],
 [4, 7],
 [5, 55],
 [6, 27],
 [7, 22],
 [8, 48],
 [9, 19],
 [10, 66],
 [11, 58],
 [12, 57],
 [13, 20],
 [14, 14],
 [15, 1],
 [16, 53],
 [17, 35],
 [18, 37],
 [19, 32],
 [20, 38],
 [21, 63],
 [22, 30],
 [23, 10],
 [24, 39],
 [25, 43],
 [26, 18],
 [27, 46],
 [28, 9],
 [29, 54],
 [30, 17],
 [31, 26],
 [32, 15],
 [33, 3],
 [34, 41],
 [35, 13],
 [36, 36],
 [37, 5],
 [38, 28],
 [39, 60],
 [40, 24],
 [41, 61],
 [42, 47],
 [43, 23],
 [44, 64],
 [45, 2],
 [46, 67],
 [47, 11],
 [48, 8],
 [49, 33],
 [50, 59],
 [51, 21],
 [52, 50],
 [53, 45],
 [54, 40],
 [55, 25],
 [56, 0],
 [57, 51],
 [58, 56],
 [59, 6],
 [60, 4],
 [61, 42],
 [62, 29],
 [63, 34],
 [64, 49],
 [65, 44],
 [66, 62],
 [67, 16]]

# Minimum Regret

In [13]:
def MR(Show_infor = False): 
    # 优化方向：（附带稳定性）最小化最大rank
    
    SMTI = Model(sense=MINIMIZE, solver_name=GRB)
        
    num_female = 68
    num_male = 68

    I = set(range(num_female))
    J = set(range(num_male))

    x = [[SMTI.add_var(var_type=BINARY) for j in J] for i in I] # Decision Variable
    z = SMTI.add_var()

    SMTI.objective = z # Objective Function


    for i in I:
        SMTI += xsum(x[i][j] for j in W_dict[i]) <= 1 

    for j in J:
        SMTI += xsum(x[i][j] for i in M_dict[j]) <= 1    


    for i in I:
        for j in W_dict[i]:
            SMTI += 1 - xsum(x[i][q] for q in w_to_m_better_or_indiff_than[i][j]) <= xsum(x[p][j] for p in m_to_w_better_or_indiff_than[j][i])

    for i in I:
        for j in W_dict[i]:
            SMTI += m_to_w_rank[j][i]*x[i][j] <= z
            SMTI += w_to_m_rank[i][j]*x[i][j] <= z
    
    start = time.perf_counter()
    status = SMTI.optimize(max_seconds=600)
    end = time.perf_counter()
    runningtime = end-start

    if Show_infor == True:
        print('optimal solution score {} found'.format(SMTI.objective_value))
        print('Best Possible:', SMTI.objective_bound)
        print('model has {} vars, {} constraints and {} nzs'.format(SMTI.num_cols, SMTI.num_rows, SMTI.num_nz))

    solution = []
    
    
    for i in I:
        for j in J:
            if x[i][j].x > 0.99:
                solution.append([i,j])
    print('runningtime',runningtime)                
    return solution, SMTI.objective_value # 匹配结果，模型最优值

In [14]:
MR(Show_infor = False)[0]

runningtime 0.749356500000431


[[0, 3],
 [1, 63],
 [2, 52],
 [3, 13],
 [4, 22],
 [5, 55],
 [6, 17],
 [7, 44],
 [8, 48],
 [9, 15],
 [10, 66],
 [11, 58],
 [12, 62],
 [13, 20],
 [14, 53],
 [15, 1],
 [16, 4],
 [17, 18],
 [18, 37],
 [19, 32],
 [20, 38],
 [21, 29],
 [22, 14],
 [23, 67],
 [24, 34],
 [25, 43],
 [26, 16],
 [27, 10],
 [28, 9],
 [29, 54],
 [30, 51],
 [31, 26],
 [32, 61],
 [33, 2],
 [34, 41],
 [35, 39],
 [36, 36],
 [37, 12],
 [38, 28],
 [39, 60],
 [40, 5],
 [41, 31],
 [42, 59],
 [43, 23],
 [44, 64],
 [45, 47],
 [46, 46],
 [47, 11],
 [48, 8],
 [49, 33],
 [50, 24],
 [51, 0],
 [52, 50],
 [53, 45],
 [54, 49],
 [55, 25],
 [56, 7],
 [57, 27],
 [58, 56],
 [59, 6],
 [60, 65],
 [61, 42],
 [62, 21],
 [63, 19],
 [64, 35],
 [65, 40],
 [66, 57],
 [67, 30]]

# SESM

In [15]:
def SESM(Show_infor = False):
    # 优化方向：（附带稳定性）最小化男女公平差异
    
    SMTI = Model(sense=MINIMIZE, solver_name=GRB)
        
    num_female = 68
    num_male = 68

    I = set(range(num_female))
    J = set(range(num_male))

    x = [[SMTI.add_var(var_type=BINARY) for j in J] for i in I] # Decision Variable
    z = SMTI.add_var()

    SMTI.objective = z # Objective Function


    for i in I:
        SMTI += xsum(x[i][j] for j in W_dict[i]) <= 1 

    for j in J:
        SMTI += xsum(x[i][j] for i in M_dict[j]) <= 1    


    for i in I:
        for j in W_dict[i]:
            SMTI += 1 - xsum(x[i][q] for q in w_to_m_better_or_indiff_than[i][j]) <= xsum(x[p][j] for p in m_to_w_better_or_indiff_than[j][i])

    SMTI += xsum((m_to_w_rank[j][i] - w_to_m_rank[i][j])*x[i][j] for i in I for j in W_dict[i]) <= z
    
    SMTI += xsum((w_to_m_rank[i][j] - m_to_w_rank[j][i])*x[i][j] for i in I for j in W_dict[i]) <= z
    
    start = time.perf_counter()
    status = SMTI.optimize(max_seconds=600)
    end = time.perf_counter()
    runningtime = end-start

    if Show_infor == True:
        print('optimal solution score {} found'.format(SMTI.objective_value))
        print('Best Possible:', SMTI.objective_bound)
        print('model has {} vars, {} constraints and {} nzs'.format(SMTI.num_cols, SMTI.num_rows, SMTI.num_nz))

    
    solution = []
    
    
    for i in I:
        for j in J:
            if x[i][j].x > 0.99:
                solution.append([i,j])
                
    print('runningtime',runningtime)                
    return solution, SMTI.objective_value # 匹配结果，模型最优值

In [16]:
SESM(Show_infor = True)[0]

optimal solution score 70.0 found
Best Possible: 70.0
model has 4625 vars, 4560 constraints and 319164 nzs
runningtime 1.071756400000595


[[0, 12],
 [1, 63],
 [2, 52],
 [3, 23],
 [4, 22],
 [5, 55],
 [6, 17],
 [7, 35],
 [8, 48],
 [9, 15],
 [10, 66],
 [11, 58],
 [12, 62],
 [13, 20],
 [14, 53],
 [15, 1],
 [16, 4],
 [17, 18],
 [18, 37],
 [19, 32],
 [20, 38],
 [21, 39],
 [22, 14],
 [23, 5],
 [24, 3],
 [25, 43],
 [26, 16],
 [27, 10],
 [28, 9],
 [29, 54],
 [30, 51],
 [31, 26],
 [32, 61],
 [33, 59],
 [34, 41],
 [35, 13],
 [36, 36],
 [37, 19],
 [38, 28],
 [39, 60],
 [40, 29],
 [41, 6],
 [42, 65],
 [43, 2],
 [44, 64],
 [45, 47],
 [46, 46],
 [47, 11],
 [48, 8],
 [49, 33],
 [50, 24],
 [51, 30],
 [52, 50],
 [53, 45],
 [54, 7],
 [55, 25],
 [56, 67],
 [57, 27],
 [58, 56],
 [59, 31],
 [60, 0],
 [61, 42],
 [62, 21],
 [63, 34],
 [64, 49],
 [65, 40],
 [66, 57],
 [67, 44]]

In [18]:
MAX_WT_solution     =  MIP_Solver(Optimize_Direction = 'MAXIMIZE', Additional_Const = 'unstable', Show_infor=True, OF = 'Maximize Weight' )[0]
MAX_WT_stable_solution =  MIP_Solver(Optimize_Direction = 'MAXIMIZE', Additional_Const = 'stable',Show_infor=True)[0]
Egalitarian_solution= MIP_Solver(Optimize_Direction = 'MINIMIZE', Additional_Const = 'stable', OF = 'Egalitarian',Show_infor=True)[0]
MR_solution         = MR(Show_infor=True)[0]
SESM_solution       = SESM(Show_infor=True)[0]

optimal solution score 3295.035193470071 found
Best Possible: 3295.0351934700716
model has 4624 vars, 136 constraints and 8844 nzs
runningtime 0.036986300000535266
optimal solution score 3220.737435583144 found
Best Possible: 3220.737435583144
model has 4624 vars, 4558 constraints and 310628 nzs
runningtime 0.49758419999943726
optimal solution score 777.0 found
Best Possible: 777.0
model has 4624 vars, 4558 constraints and 310628 nzs
runningtime 0.914850000000115
optimal solution score 28.0 found
Best Possible: 28.0
model has 4625 vars, 13402 constraints and 328316 nzs
runningtime 0.7577535999998872
optimal solution score 70.0 found
Best Possible: 70.0
model has 4625 vars, 4560 constraints and 319164 nzs
runningtime 1.0955309999999372


In [19]:
MAX_WT_solution

[[0, 12],
 [1, 31],
 [2, 52],
 [3, 65],
 [4, 7],
 [5, 55],
 [6, 27],
 [7, 22],
 [8, 48],
 [9, 19],
 [10, 66],
 [11, 58],
 [12, 57],
 [13, 20],
 [14, 14],
 [15, 1],
 [16, 53],
 [17, 35],
 [18, 37],
 [19, 32],
 [20, 38],
 [21, 63],
 [22, 30],
 [23, 10],
 [24, 39],
 [25, 43],
 [26, 18],
 [27, 46],
 [28, 9],
 [29, 54],
 [30, 17],
 [31, 26],
 [32, 15],
 [33, 3],
 [34, 41],
 [35, 13],
 [36, 36],
 [37, 5],
 [38, 28],
 [39, 60],
 [40, 24],
 [41, 61],
 [42, 47],
 [43, 23],
 [44, 64],
 [45, 2],
 [46, 67],
 [47, 11],
 [48, 8],
 [49, 33],
 [50, 59],
 [51, 21],
 [52, 50],
 [53, 45],
 [54, 40],
 [55, 25],
 [56, 0],
 [57, 51],
 [58, 56],
 [59, 6],
 [60, 4],
 [61, 42],
 [62, 29],
 [63, 34],
 [64, 49],
 [65, 44],
 [66, 62],
 [67, 16]]